In [5]:
# !pip install elasticsearch7

# Python Client

- https://elasticsearch-py.readthedocs.io/en/latest/

In [4]:
from elasticsearch7 import Elasticsearch
import pprint
import pandas as pd
import json

HOST='localhost'
URL_PREFIX = 'http://{host}:9200'.format(host=HOST)
# Kibana: http://localhost:5601/app/management/security/api_keys/
API_KEY = 'ank1Q0xKRUJCa1JiYXI5Y1V2ZXU6S2MzQ0JPZVVRYW1abGtwQzVGaWhSZw=='

client = Elasticsearch(hosts=URL_PREFIX, api_key=API_KEY)
resp = client.info()
pprint.pprint(resp)
pd.json_normalize(resp)

{'cluster_name': 'es-cluster',
 'cluster_uuid': 'OyYooDV_RrWxLxSvpxmoLw',
 'name': 'es-node',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2023-06-23T05:33:12.261262042Z',
             'build_flavor': 'default',
             'build_hash': 'eeedb98c60326ea3d46caef960fb4c77958fb885',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '8.11.1',
             'minimum_index_compatibility_version': '6.0.0-beta1',
             'minimum_wire_compatibility_version': '6.8.0',
             'number': '7.17.11'}}


,name,cluster_name,cluster_uuid,tagline,version.number,version.build_flavor,version.build_type,version.build_hash,version.build_date,version.build_snapshot,version.lucene_version,version.minimum_wire_compatibility_version,version.minimum_index_compatibility_version
0,es-node,es-cluster,OyYooDV_RrWxLxSvpxmoLw,"You Know, for Search",7.17.11,default,docker,eeedb98c60326ea3d46caef960fb4c77958fb885,2023-06-23T05:33:12.261262042Z,False,8.11.1,6.8.0,6.0.0-beta1


# Local Maven Repository

## Read Respository

In [2]:
from pathlib import Path
import pandas as pd
from datetime import datetime

# verify with shell: find . | grep -E 'jar$' | wc -l
def list_artifact(starting_directory : str, glob="*") -> list[str]:
    poms = []
    path_object = Path(starting_directory)
    for file_path in path_object.rglob(glob):
        if file_path.is_file():
            stat = file_path.stat()
            p = file_path.as_posix()
            if p.find('-sources.jar') != -1: # skip sources
                continue
            if p.find('SNAPSHOT') != -1 and not p.endswith('-SNAPSHOT.jar'): # skip sources
                continue
            artifacts = p.removeprefix(starting_directory).split('/')
            # print(artifacts)
            poms.append({
                # 'folder': file_path.parent.as_posix(),
                # 'path': artifacts,
                'group': '.'.join(artifacts[:-3]),
                'artifact':  artifacts[-3],
                'version': artifacts[-2],
                'size': stat.st_size, # / 1024 / 1024, # MB
                'ctime': datetime.fromtimestamp(stat.st_ctime).strftime('%Y-%m-%d %H:%M:%S.%f'),
                'jar': artifacts[-1]
            })
    return poms

artifact_home = "C:/Users/zhouj/.m2/repository/"
artifacts = list_artifact(artifact_home, "*.jar")
# pd.json_normalize(artifacts)
print(len(artifacts))
s = pd.json_normalize(artifacts)
# s.sort_values(by=['size', 'group', 'artifact'], ascending=False)
# s['path.size'].sum()

s.sort_values(by=['ctime'], ascending=True)

2592


,group,artifact,version,size,ctime,jar
1334,org.apache.maven.plugins,maven-clean-plugin,3.2.0,35678,2024-03-07 12:43:26.969265,maven-clean-plugin-3.2.0.jar
1416,org.apache.maven.shared,maven-shared-utils,3.3.4,153143,2024-03-07 12:43:36.989123,maven-shared-utils-3.3.4.jar
500,commons-io,commons-io,2.6,214788,2024-03-07 12:43:42.467580,commons-io-2.6.jar
1355,org.apache.maven.plugins,maven-resources-plugin,3.3.0,31760,2024-03-07 12:44:03.833816,maven-resources-plugin-3.3.0.jar
1336,org.apache.maven.plugins,maven-compiler-plugin,3.10.1,61899,2024-03-07 12:44:05.852205,maven-compiler-plugin-3.10.1.jar
...,...,...,...,...,...,...
2163,org.junit.platform,junit-platform-launcher,1.3.1,94645,2024-08-06 09:08:39.365935,junit-platform-launcher-1.3.1.jar
1445,org.apache.maven.surefire,surefire-junit-platform,2.22.2,65915,2024-08-06 09:08:39.866519,surefire-junit-platform-2.22.2.jar
1611,org.apiguardian,apiguardian-api,1.0.0,2164,2024-08-06 09:08:40.663393,apiguardian-api-1.0.0.jar
2234,org.opentest4j,opentest4j,1.1.1,7121,2024-08-06 09:08:49.388928,opentest4j-1.1.1.jar


## mapping

In [12]:
INDEX = 'maven'

mappings = {
  "properties": {
    "artifact": {
      "type": "text",
      "fields": {
        "keyword": {
          "type": "keyword",
          "ignore_above": 256
        }
      }
    },
    "ctime": { # https://www.elastic.co/guide/en/elasticsearch/reference/7.17/date.html
      "type": "date",
       "format": "yyyy-MM-dd HH:mm:ss||yyyy-MM-dd||epoch_millis||yyyy-MM-dd HH:mm:ss.SSSSSS"
    },
    "group": {
      "type": "text",
      "fields": {
        "keyword": {
          "type": "keyword",
          "ignore_above": 256
        }
      }
    },
    "size": {
      "type": "long"
    },
    "version": {
      "type": "text",
      "fields": {
        "keyword": {
          "type": "keyword",
          "ignore_above": 256
        }
      }
    },
    "jar": {
      "type": "text",
      "fields": {
        "keyword": {
          "type": "keyword",
          "ignore_above": 256
        }
      }
    }
  }
}

res = client.indices.delete(index=INDEX) # delete first
print(res)
client.indices.create(index=INDEX, mappings=mappings)

{'acknowledged': True}


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'maven'}

## Looped Index

In [18]:
# slow
for artifact in artifacts:
    _id = artifact['group'] + ':' + artifact['artifact'] + ':' + artifact['version']
    client.index(index=INDEX,
        id=_id,
        document=artifact,)

## Bulk Index

In [25]:
# help(client.bulk)
# from elasticsearch7 import helpers
# help(helpers.bulk)

In [8]:
ids = [artifact['jar'][:-4] for artifact in artifacts]
len(ids), len(set(ids))

(2592, 2592)

In [13]:
_body = ""
for i in range(len(artifacts)):
    _id = artifacts[i]['jar'][:-4]
    _body  = _body + """
{action}
{doc}""".format(action=json.dumps({'index': {"_id" : _id, "_index": INDEX}}), doc=json.dumps(artifacts[i]))
    
    if i > 0 and i % 100 == 0:
        print(f"Progress: {i}")
        res = client.bulk(index=INDEX, body=_body)
        # print(res)
        _body = ""
        

if len(_body) > 0:
    print(f"Progress: {i}")
    res = client.bulk(index=INDEX, body=_body)
    # print(res)

Progress: 100
Progress: 200
Progress: 300
Progress: 400
Progress: 500
Progress: 600
Progress: 700
Progress: 800
Progress: 900
Progress: 1000
Progress: 1100
Progress: 1200
Progress: 1300
Progress: 1400
Progress: 1500
Progress: 1600
Progress: 1700
Progress: 1800
Progress: 1900
Progress: 2000
Progress: 2100
Progress: 2200
Progress: 2300
Progress: 2400
Progress: 2500
Progress: 2591
{'took': 22, 'errors': False, 'items': [{'index': {'_index': 'maven', '_type': '_doc', '_id': 'spring-expression-5.3.23', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2501, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'maven', '_type': '_doc', '_id': 'spring-expression-5.3.31', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2502, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'maven', '_type': '_doc', '_id': 'spring-expression-6.1.5', '_version': 1, 'result': 'created', '_shards

In [14]:
res = client.search(index="maven", query={"match": {"group": {"query": "org.rocksdb"}}})
print(res)
pd.json_normalize(res['hits']['hits'])

{'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'}, 'max_score': 7.0919447, 'hits': [{'_index': 'maven', '_type': '_doc', '_id': 'rocksdbjni-7.0.3', '_score': 7.0919447, '_source': {'group': 'org.rocksdb', 'artifact': 'rocksdbjni', 'version': '7.0.3', 'size': 54269203, 'ctime': '2024-04-10 10:55:52.766078', 'jar': 'rocksdbjni-7.0.3.jar'}}, {'_index': 'maven', '_type': '_doc', '_id': 'rocksdbjni-9.4.0', '_score': 7.0919447, '_source': {'group': 'org.rocksdb', 'artifact': 'rocksdbjni', 'version': '9.4.0', 'size': 70111775, 'ctime': '2024-08-07 11:15:18.763036', 'jar': 'rocksdbjni-9.4.0.jar'}}]}}


,_index,_type,_id,_score,_source.group,_source.artifact,_source.version,_source.size,_source.ctime,_source.jar
0,maven,_doc,rocksdbjni-7.0.3,7.091945,org.rocksdb,rocksdbjni,7.0.3,54269203,2024-04-10 10:55:52.766078,rocksdbjni-7.0.3.jar
1,maven,_doc,rocksdbjni-9.4.0,7.091945,org.rocksdb,rocksdbjni,9.4.0,70111775,2024-08-07 11:15:18.763036,rocksdbjni-9.4.0.jar


## SQL

In [21]:
# help(client)
res = client.sql.query(body={
  'query': """SELECT "group", artifact, version, COUNT(*) as c
  FROM maven
  GROUP BY "group", artifact, version
  HAVING c > 1
  ORDER BY c DESC"""
}, format='txt')
print(res)

           group            |           artifact            |    version    |       c       
----------------------------+-------------------------------+---------------+---------------
org.bytedeco                |javacpp                        |1.5.6          |16             
org.bytedeco                |artoolkitplus                  |2.3.1-1.5.9    |13             
org.bytedeco                |ffmpeg                         |4.4-1.5.6      |13             
org.bytedeco.javacpp-presets|ffmpeg                         |4.0.2-1.4.3    |12             
org.bytedeco                |javacpp                        |1.5.10         |11             
org.bytedeco                |openblas                       |0.3.26-1.5.10  |11             
org.bytedeco                |opencv                         |4.9.0-1.5.10   |11             
org.bytedeco                |ffmpeg                         |6.1.1-1.5.10   |9              
org.bytedeco                |leptonica                      |1.84.1-1.